In [1]:
import numpy as np
import matplotlib.pyplot as plt

ImportError: No module named matplotlib.pyplot

In [2]:
# Parameters
hidden_layers = 2
nodes_in_layer = [10, 5]
number_of_training_iterations = 20000

In [3]:
# Get Data
# Data loading

my_data = np.genfromtxt('XOR2_16nm_stat00.csv', delimiter=',',skip_header=1,dtype = 'float')
data = my_data
# data = np.genfromtxt(cStringIO.StringIO(data),dtype = "float", delimiter=",", names = True)

IOError: XOR2_16nm_stat00.csv not found.

In [4]:
# import cStringIO
# with open ("XOR2_16nm_stat00.csv", "r") as myfile:
#        data = myfile.read().replace('"', '')

In [ ]:
print(data.shape)

In [ ]:
# length = data.shape[0]
# feature_count = data.shape[1]
# 
# train_data = data

# train_data_inputs = train_data[:, :feature_count - 3]
# train_data_outputs = train_data[:, feature_count - 3: feature_count]

In [ ]:
# Test_Sampling
# train_sample_input = train_data_inputs[0:int((0.8) * data.shape[0])]
# train_sample_output = train_data_outputs[0:int((0.8) * data.shape[0])]

# test_sample_input = train_data_inputs[int((0.8) * data.shape[0]) : data.shape[0]]
# test_sample_output = train_data_outputs[int((0.8) * data.shape[0]) : data.shape[0]]
Train = my_data[1:3750,:]
Test  = my_data[3750:5000,:]
Train_X = Train[:,0:20]
Train_y =Train[:,20:23]
Test_X = Test[:,0:20]
Test_y =  Test[:,20:23]
# print(Train)

In [ ]:
print(Train_y.shape)
print(Test_y.shape)
print(Test_y[1200][2])

In [ ]:
# Prep Output data
# train_sample_output_vector = np.asarray(prep_output(train_sample_output, 10))

In [ ]:
# Pre Process Data
def pre_process(train_sample_input):

    # Remove unncessary features
    temp = train_sample_input
    final = []
    mini = [temp.min(axis=0)]
    maxi = [temp.max(axis=0)]
    diff = np.subtract(maxi, mini)
    remove_idx = np.where(diff == 0)[1]
    new_temp = np.delete(temp, remove_idx, 1)

    # Normalize
    mini = [new_temp.min(axis=0)]
    maxi = [new_temp.max(axis=0)]
    diff = np.subtract(maxi, mini)
    for row in new_temp:
        row = (np.subtract(row, mini))/(np.subtract(maxi, mini) + 1)
        final.append(row)
    preproc_train_sample = np.asarray(final)[:,0,:]
    return preproc_train_sample

In [ ]:
def prep_output(y, no_of_classes):
    output = []
    for i in y:
        arr = np.zeros(no_of_classes)
        arr[int(i)] = 1
        output.append(arr)
    return output

In [ ]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [ ]:
def sigmoid_der(x):
    return x*(1-x)

In [ ]:
 def ReLU(x):
    x[x < 0] = 0
    return x

In [ ]:
def ReLU_der(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [ ]:
def tanh(x):
    return np.tanh(x)

In [ ]:
def tanh_der(x):
    return 1.0 - np.tanh(x)**2

In [ ]:
def softmax(x):
    return np.exp(x.T) / np.sum(np.exp(x.T), axis=0)

In [ ]:
def cross_entropy(predictions, targets, epsilon=1e-12):
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/N
    return ce

In [ ]:
# Initialize Layer Weights:
def init_weights(hidden_layers, nodes_in_layer, input_count, output_count=3):
    weight_array = []
    np.random.seed(2)
    for i in range(hidden_layers + 1):
        if(i == 0):
            weight_array.append(2 * np.random.random((input_count, nodes_in_layer[i])) - 1)
        elif(i == hidden_layers):
            weight_array.append(2 * np.random.random((nodes_in_layer[i - 1], output_count)) - 1)
        else:
            weight_array.append(2 * np.random.random((nodes_in_layer[i - 1], nodes_in_layer[i])) - 1)

    return weight_array

In [ ]:
def pad_input_array(x):
    n = x.shape[0]
    m = x.shape[1]
    padded_array = np.zeros(shape=(n, (m + 1))) + 1
    padded_array[:,1:] = x
    return padded_array

In [ ]:
def train(hidden_layers_count, nodes_in_layer, number_of_training_iterations, input_data, output_data, alpha=8 * 10e-5, activation="tanh"):
    loss_function = []
    n = hidden_layers_count + 1
    input_count = input_data.shape[1]
    output_count = output_data.shape[1]
    weight_arr = init_weights(hidden_layers_count, nodes_in_layer, input_count)
    output = []
    for m in range(number_of_training_iterations + 1):
        
        layer_output = []
        layer_output.append(input_data)
        for i in range(0, n):
            if(activation == "sigmoid"):
                layer_output.append(sigmoid(np.dot(layer_output[i], weight_arr[i])))
            elif(activation == "ReLU"):
                layer_output.append(ReLU(np.dot(layer_output[i], weight_arr[i])))
            elif(activation == "tanh"):
                layer_output.append(tanh(np.dot(layer_output[i], weight_arr[i])))
        layer_output.append(tanh(np.dot(layer_output[i], weight_arr[hidden_layers_count])))
        if(m == number_of_training_iterations):
            output = layer_output
            f=open('weights_test.out','ab')
            for data_slice in weight_arr:
                np.savetxt(f, data_slice)
            break

        delta_arr = [None] * n
        error_arr = [None] * n
        for i in range(hidden_layers_count, -1, -1):
            if(i == hidden_layers_count):
                error_arr[i] = output_data - layer_output[i + 2]
            else:
                error_arr[i] = delta_arr[i + 1].dot(weight_arr[i + 1].T)
            if(activation == "sigmoid"):
                delta_arr[i] = error_arr[i] * sigmoid_der(layer_output[i + 1])
            elif(activation == "ReLU"):
                delta_arr[i] = error_arr[i] * ReLU_der(layer_output[i + 1])
            elif(activation == "tanh"):
                delta_arr[i] = error_arr[i] * tanh_der(layer_output[i + 1])

        loss_function.append(cross_entropy(layer_output[hidden_layers_count + 1], output_data))
        if(m%100 == 0):
            print("Error: ", str(np.mean(np.abs(error_arr[hidden_layers_count]))))
            
        for i in range(0, n):
            weight_arr[i] += alpha * layer_output[i].T.dot(delta_arr[i])
    
    plt.plot(range(1, number_of_training_iterations + 1), loss_function)
    return output[hidden_layers_count + 1], weight_arr

In [ ]:
# Softmax the output
def final_output(inp):
    softmax_op = tanh(inp).T
    length_op = len(softmax_op)
    
    
    output_list = [softmax_op[i].argmax(axis=0) for i in range(length_op)]
    return output_list

In [ ]:
# Preproc subset with varying iteration count
# preproc_train_example = pre_process(train_sample_input)
# padded_input_example = pad_input_array(preproc_train_example)


weight_arr = train(hidden_layers, nodes_in_layer, 10000, Train_X, Train_y,alpha= 10e-5, activation="tanh")

0.0017344515001842286 Sigmoid 15 
-9.989999933407882e-10 ReLU 15
0.23845533036627764 tanh 15


Error:  0.16577812970189867 ReLU 18 500_vals

In [ ]:
print(np.asarray(np.where(train_sample_output - final_output(output[0]) == 0)).shape(1))
print(train_sample_output.shape[0])

In [ ]:
plt.plot(range(1,5), error_list)

In [ ]:
def predict(weight_arr, input_data, hidden_layers_count, activation="tanh"):
    layer_output = []
    layer_output.append(input_data)
    for i in range(0, hidden_layers_count + 1):
        layer_output.append(tanh(np.dot(layer_output[i], weight_arr[i])))
    layer_output.append(np.asarray(tanh(layer_output[hidden_layers_count + 1])).T)
    return layer_output[hidden_layers_count + 1]

In [ ]:
#validation_data = np.genfromtxt("Apparel/apparel-trainval.csv", delimiter=',', skip_header=1)

pad_data = pad_input_array(Test_X)
final = predict(weight_arr, pad_data, 1, activation="tanh")

In [ ]:
np.savetxt("test_output_final_final_OP_new.csv", final_output(final), newline=",")

In [ ]:
print(np.asarray(np.where(train_sample_output[0:300] - final_output(predict(weight_arr[1], data[0:300], 1, activation="sigmoid")) == 0)).shape[1])

In [ ]:
#DO NOT RUN
op_1 = weight_arr

In [ ]:
print(np.asarray(np.where(train_sample_output[0:500] - final_output(predict(op_1[1], pad_input_array(train_sample_input[0:500]), 1, activation="sigmoid")) == 0)).shape[1])

In [ ]:
print(data.shape)

In [ ]:
# DO NOT RUN - 2
op_2 = weight_arr

In [ ]:
print(np.asarray(np.where(train_sample_output - final_output(predict(weight_arr[1], pad_input_array(train_sample_input), 1, activation="sigmoid")) == 0)).shape[1])

In [ ]:
final_prediction = final_output(predict(weight_arr[1], pad_input_array(train_sample_input), 1, activation="sigmoid"))
np.savetxt("20161116_prediction.csv", final_prediction, delimiter=",")

In [ ]:
temp = np.asarray(train_sample_output - final_output(predict(weight_arr[1], pad_input_array(train_sample_input), 1, activation="sigmoid")))
print(temp)

In [ ]:
t = [[1 , 2 , 3, 4], [1 , 2 , 3, 4], [1 , 2 , 3, 4]]
t= np.asarray(t)
print(t.sum(axis = 1))

In [ ]:
import pandas as pd
final_op = np.asarray(final_output(predict(weight_arr[1], pad_input_array(train_sample_input), 1, activation="sigmoid")))
print(final_op)
# pd.DataFrame(final_op.T).to_csv("20161116_prediction_2.csv", header=None, index=None)
np.savetxt("20161116_prediction_3.csv", final_op.astype(int), newline=",")